# Demo of Meteorological Drought Index
* Some of the meteorological drought index computation is modified from the existing python code provided in [here](https://www.drought.gov/drought/climate-and-drought-indices-python). Please check the link to understand the concept and math behind the code.


In [1]:
import pandas as pd
import numpy as np

In [2]:
from climate_indices.indices import spi, spei, percentage_of_normal

# create class for computing montly climate drought indecies
class monthly_climate_indices():
    
    def __init__(self, weather_data):
        """
        In this example, weather data is ordered by year and months.
        To set up the temporal running window, each monthly index uses the information
        three months ago. The first 5 year computation is used to calibration.
        """
        self.start_year = weather_data['Year'].values[0]
        self.final_year_cal = self.start_year + 5
        self.scale = 5 - weather_data['Mn'].values[0]
        
        self.mn_ppt = weather_data['ppt_sum'].values * 25.4 # convert to mm
        self.tmean = (weather_data['tmean'].values - 32) * 5 / 9 # convert to C
        
        
    @staticmethod
    def consecutive_chunk(dat, chunk_size):
        '''
        create chunks with same size that is consecutive
        '''
        return [dat[i : i+chunk_size] for i in np.arange(len(dat) - chunk_size + 1)]
    
    @staticmethod
    def cumulative_chunk(dat, chunk_size):
        '''
        create chunks with increasing size that is consecutive
        '''
        return [dat[0 : i+chunk_size] for i in np.arange(len(dat) - chunk_size + 1)]

    def get_SPI(self):
        '''
        climate indecis function based on script which is modifed from:
        https://www.drought.gov/drought/climate-and-drought-indices-python
        '''
        return spi( precips=self.mn_ppt, 
                    scale=self.scale,
                    distribution='gamma',
                    data_start_year=self.start_year,
                    calibration_year_initial=self.start_year,
                    calibration_year_final=self.final_year_cal,
                    periodicity='monthly')


    def get_PET_SPEI(self, latitude_degrees):
        '''
        climate indecis function based on script which is modifed from:
        https://www.drought.gov/drought/climate-and-drought-indices-python
        '''     
        SPEI, pet_mm =  spei(scale=self.scale,
                             distribution='gamma',
                             periodicity='monthly',
                             data_start_year=self.start_year,
                             calibration_year_initial=self.start_year,
                             calibration_year_final=self.final_year_cal,
                             precips_mm=self.mn_ppt,
                             temps_celsius=self.tmean,
                             latitude_degrees=latitude_degrees)
        return pet_mm, SPEI


    def get_PNP(self):
        '''
        climate indecis function based on script which is modifed from:
        https://www.drought.gov/drought/climate-and-drought-indices-python
        '''
        return percentage_of_normal( values=self.mn_ppt,
                                     scale=self.scale,
                                     data_start_year=self.start_year,
                                     calibration_start_year=self.start_year,
                                     calibration_end_year=self.final_year_cal,
                                     periodicity='monthly')
    
    
    def get_RDI(self, latitude_degrees):
        '''
        function is build based on
        Tsakiris, G., & Vangelis, H. J. E. W. (2005). 
        Establishing a drought index incorporating evapotranspiration. 
        European water, 9(10), 3-11.
        '''
        _, pet_mm = spei(scale=self.scale,
                         distribution='gamma',
                         periodicity='monthly',
                         data_start_year=self.start_year,
                         calibration_year_initial=self.start_year,
                         calibration_year_final=self.final_year_cal,
                         precips_mm=self.mn_ppt,
                         temps_celsius=self.tmean,
                         latitude_degrees=latitude_degrees)
        
        # create consecutive chunks for calculating cumulative weather effects
        mn_ppt_chunks = self.cumulative_chunk(self.mn_ppt, 1)
        PET_chunks = self.cumulative_chunk(pet_mm, 1)
        RDI = np.array([np.nan] * (self.scale - 1)) # nan for months less than scale
        y = np.array([])
        
        for i, (ppt, PET) in enumerate(zip(mn_ppt_chunks, PET_chunks)):
                
            ppt, PET = np.array(ppt), np.array(PET)
            y_i = np.log(np.sum(ppt) / np.sum(PET))
            y = np.append(y, y_i)
            
            if i >= (self.scale - 1):
                RDI = np.append(RDI, (y_i - np.mean(y)) / np.std(y) )
                y = y[1:] # remove y for the earliest month
        
        return RDI